In [3]:
import certifi
import urllib3

In [4]:
url = 'http://www.wahlrecht.de/umfragen/allensbach.htm'

http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

r = http.request('GET', url)

In [6]:
r.data[1:500]

b'!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\n<html lang="de-DE">\n\n<head>\n<meta http-equiv="expires" content="0">\n<meta http-equiv="content-type" content="text/html; charset=UTF-8">\n<meta name="description" content="Allensbach &ndash; Aktuelle Ergebnisse von Umfragen zur Bundestagswahl (Sonntagsfrage)">\n<title>Sonntagsfrage &ndash; Allensbach</title>\n<link rel="icon" type="image/png" href="../favicon.png">\n<link rel="alternate" type="appli'

In [9]:
from bs4 import BeautifulSoup

In [10]:
data = BeautifulSoup(r.data, "lxml")

In [9]:
print(data.prettify())

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="de-DE">
 <head>
  <meta content="0" http-equiv="expires"/>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="Allensbach – Aktuelle Ergebnisse von Umfragen zur Bundestagswahl (Sonntagsfrage)" name="description"/>
  <title>
   Sonntagsfrage – Allensbach
  </title>
  <link href="../favicon.png" rel="icon" type="image/png"/>
  <link href="umfragen.xml" rel="alternate" title="Wahlrecht.de – Umfragen (RSS)" type="application/rss+xml"/>
  <link href="../normal.css" rel="stylesheet" type="text/css"/>
  <link href="../images/umfrage.css" rel="stylesheet" type="text/css"/>
  <style type="text/css">
   <!--
h2:target, h3:target, h4:target, ul:target, li:target, p:target, span:target, table:target, col:target, colgroup:target, tr:target, th:target, td:target { background: #e0e0ff; }
-->
  </style>
 </head>
 <body>
  <div class="head">
   <table cl

Extracting table content:

In [11]:
main_table = data.find('table', class_='wilko')

In [11]:
main_table

<table align="center" border="0" cellpadding="2" cellspacing="3" class="wilko">
<thead>
<tr>
<th class="dat"> </th>
<th> </th>
<th class="part"><a href="#fn-cdu">CDU</a>/<a href="#fn-csu">CSU</a></th>
<th class="part"><a href="#fn-spd">SPD</a></th>
<th class="part"><a href="#fn-gru">GRÜNE</a></th>
<th class="part"><a href="#fn-fdp">FDP</a></th>
<th class="part"><a href="#fn-lin">LINKE</a></th>
<th class="part"><a href="#fn-afd">AfD</a></th>
<th><a href="#fn-son">Sonstige</a></th>
<th> </th>
<th class="befr">Befragte</th>
<th class="dat2">Zeitraum</th>
</tr>
</thead>
<tfoot>
<tr>
<th class="dat"> </th>
<th> </th>
<th class="part"><a href="#fn-cdu">CDU</a>/<a href="#fn-csu">CSU</a></th>
<th class="part"><a href="#fn-spd">SPD</a></th>
<th class="part"><a href="#fn-gru">GRÜNE</a></th>
<th class="part"><a href="#fn-fdp">FDP</a></th>
<th class="part"><a href="#fn-lin">LINKE</a></th>
<th class="part"><a href="#fn-afd">AfD</a></th>
<th><a href="#fn-son">Sonstige</a></th>
<th> </th>
<th class="

In [12]:
rows = main_table.find_all('tr')

In [35]:
test = rows[6].find_all('td')

In [41]:
[x.text for x in test if x.text.strip() != '']

['18.07.2017',
 '39,5 %',
 '25,0 %',
 '7,0 %',
 '9,0 %',
 '9,0 %',
 '7,0 %',
 '3,5 %',
 '1.403',
 '01.07.–12.07.']

In [40]:
test[1].text.strip()

''

### Read table with pandas

In [1]:
import pandas as pd

In [148]:
table = pd.read_html(r.data)[1]

In [149]:
table.tail()

,Unnamed: 0,Unnamed: 1,CDU/CSU,SPD,GRÜNE,FDP,LINKE,AfD,Sonstige,Unnamed: 9,Befragte,Zeitraum,Unnamed: 12,Unnamed: 13
45,19.12.2013,NaN,"41,5 %","25,5 %","10,5 %","4,0 %","9,0 %","4,5 %","5,0 %",NaN,?,01.12.–12.12.,NaN,NaN
46,27.11.2013,NaN,"40,0 %","26,5 %","10,0 %","4,0 %","9,5 %","5,0 %","5,0 %",NaN,?,08.11.–21.11.,NaN,NaN
47,23.10.2013,NaN,"41,0 %","25,0 %","9,0 %","5,0 %","9,0 %","5,5 %","5,5 %",NaN,?,01.10.–16.10.,NaN,NaN
48,22.09.2013,NaN,"41,5 %","25,7 %","8,4 %","4,8 %","8,6 %","4,7 %","PIR 2,2 %Sonst. 4,0 %",NaN,Bundestagswahl,NaN,NaN,NaN
49,NaN,NaN,CDU/CSU,SPD,GRÜNE,FDP,LINKE,AfD,Sonstige,NaN,Befragte,Zeitraum,LINKE: DIE LINKE,AfD = Alternative für Deutschland CDU = Chris...


Cleaning:

In [150]:
table.drop(table.columns[[1,9,12,13]], axis=1, inplace=True)

In [151]:
table.drop([48, 49], axis=0, inplace=True)

In [152]:
table.columns.values[0] = 'Veröffentlichung'

In [153]:
table.columns.values

array(['Veröffentlichung', 'CDU/CSU', 'SPD', 'GRÜNE', 'FDP', 'LINKE',
       'AfD', 'Sonstige', 'Befragte', 'Zeitraum'], dtype=object)

Cleaning values and change data type:

In [164]:
table['Veröffentlichung'] = pd.to_datetime(table['Veröffentlichung'], format='%d.%m.%Y')

In [154]:
for col in table.columns.values[1:8]:
    table[col] = pd.to_numeric(table[col].str.replace('%', '').str.replace(',', '.'))

In [157]:
# convert numbers of people surveyed to numeric
table['Befragte'] = pd.to_numeric(table['Befragte'].str.replace('≈', '').str.replace('?', '').str.replace('.', ''))

In [166]:
table['Institut'] = 'Allensbach'

In [167]:
table

,Veröffentlichung,CDU/CSU,SPD,GRÜNE,FDP,LINKE,AfD,Sonstige,Befragte,Zeitraum,Institut
0,2017-09-06,38.5,24.0,7.5,10.0,8.0,8.0,4.0,1043.0,22.08.–31.08.,Allensbach
1,2017-08-22,39.5,24.0,7.5,10.0,8.0,7.0,4.0,1421.0,04.08.–17.08.,Allensbach
2,2017-07-18,39.5,25.0,7.0,9.0,9.0,7.0,3.5,1403.0,01.07.–12.07.,Allensbach
3,2017-06-20,40.0,24.0,7.0,10.5,8.5,6.5,3.5,1437.0,01.06.–15.06.,Allensbach
4,2017-05-26,37.0,26.0,8.0,9.0,8.0,8.0,4.0,1457.0,05.05.–19.05.,Allensbach
5,2017-04-25,36.0,31.0,7.0,6.0,9.0,7.0,4.0,1407.0,01.04.–13.04.,Allensbach
6,2017-03-28,34.0,33.0,7.5,6.5,8.0,7.0,4.0,1397.0,06.03.–19.03.,Allensbach
7,2017-02-22,33.0,30.5,8.0,7.0,8.0,8.5,5.0,1542.0,01.02.–15.02.,Allensbach
8,2017-01-26,36.0,23.0,9.0,7.0,9.5,11.5,4.0,1441.0,05.01.–19.01.,Allensbach
9,2016-12-22,35.5,22.0,10.0,7.5,9.5,10.5,5.0,1459.0,01.12.–15.12.,Allensbach


In [115]:
table.tail()

,Veröffentlichung,CDU/CSU,SPD,GRÜNE,FDP,LINKE,AfD,Sonstige,Befragte,Zeitraum
43,20.02.2014,40.0,26.5,10.5,4.0,9.0,5.0,5.0,?,01.02.–13.02.
44,29.01.2014,40.0,26.0,10.0,5.0,8.5,5.0,5.5,?,10.01.–23.01.
45,19.12.2013,41.5,25.5,10.5,4.0,9.0,4.5,5.0,?,01.12.–12.12.
46,27.11.2013,40.0,26.5,10.0,4.0,9.5,5.0,5.0,?,08.11.–21.11.
47,23.10.2013,41.0,25.0,9.0,5.0,9.0,5.5,5.5,?,01.10.–16.10.
